# Installing Package dan Importing Module

In [1]:
# Installing this package to connected our python file with our dataset that we already have in mysql
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 54.7 MB/s eta 0:00:00


In [2]:
# Importing mysql.connector module that's been installed
import mysql.connector

In [3]:
# Importing pandas module that used for processing dataframe (in here is our dataset which is in csv format)
import pandas as pd

In [4]:
# Installing keras-tuner package
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.8 MB/s eta 0:00:00


In [5]:
# Importing tensorflow
import tensorflow as tf

In [6]:
# Importing modul from tensorflow.keras to pre-processing text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [7]:
# Importing modul from tensorflow.keras to make model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam

In [9]:
# Importing regex module that used for pre-processing (cleaning text data)
import re

In [10]:
# Importing pandas module that used for processing dataframe (in here is our dataset which is in csv format)
import pandas as pd

In [13]:
# Importing numpy module to provide mathematical operations in arrays
import numpy as np

In [12]:
# Importing nltk module that used for pre-processing (cleaning text data)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

# Dataset

In [14]:
# Replace the placeholders with your own values
connection_name = 'c23df02-diskusai:asia-southeast2:diskusaidataset'
ip_address = '34.101.70.233'
db_name = 'diskusaidataset'
user = 'root'
password = 'df02'

# Create the connection string
connection_string = f"mysql+mysqlconnector://{user}:{password}@{ip_address}/{db_name}?unix_socket=/cloudsql/{connection_name}"

# Establish the connection
cnx = mysql.connector.connect(user=user, password=password, host=ip_address, database=db_name)


In [15]:
# Replace 'your_table_name' with the actual name of your table
query = "SELECT * FROM diskusaidataset"

# Execute the query
cursor = cnx.cursor()
cursor.execute(query)

# Fetch all the results
data = cursor.fetchall()

# Convert the data into a Pandas DataFrame
dataset = pd.DataFrame(data, columns=cursor.column_names)

# Close the cursor and connection
cursor.close()
cnx.close()


In [16]:
# Cleaning question column from special characters that don't exist in regex module
dataset['question'] = dataset['question'].str.replace('<p>', '')
dataset['question'] = dataset['question'].str.replace('<h2>', '')
dataset['question'] = dataset['question'].str.replace('</h2>', '')
dataset['question'] = dataset['question'].str.replace('<h3>', '')
dataset['question'] = dataset['question'].str.replace('</h3>', '')
dataset['question'] = dataset['question'].str.replace('</p>', '')
dataset['question'] = dataset['question'].str.replace('<strong>', '')
dataset['question'] = dataset['question'].str.replace('</strong>', '')

In [17]:
# Creating new dataset that contained only 1 column called 'sentence' consisted from "module_name", "discussion_title", "question" columns
data1=((dataset["module_name"]).values).tolist()
data2=((dataset["discussion_title"]).values).tolist()
data3=((dataset["question"]).values).tolist()
data=data1+data2+data3
df = pd.DataFrame(data, columns=['sentence'])

In [18]:
# Cleaning Data Text

STOPWORDS1 = set(stopwords.words('indonesian'))
STOPWORDS2 = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

# Defining function to clean data text using regex module
def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

# Defining tokenizer1 to clean data text using indonesian stopwords
def tokenizer1(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS1, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    stemmer = WordNetLemmatizer()
    tokens1 = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    tokens1 = ' '.join(tokens1)
    return tokens1    

# Defining tokenizer2 to clean data text using english stopwords
def tokenizer2(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS2, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    stemmer = WordNetLemmatizer()
    tokens2 = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    tokens2 = ' '.join(tokens2)
    return tokens2    

# Defining clean_sentences to apply all cleaning data text process to our dataset
def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['sentence'] = df['sentence'].apply(clean_text)
    df['sentence'] = df['sentence'].apply(
        lambda x: tokenizer1(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS1, lemmatize=True))
    df['sentence'] = df['sentence'].apply(
        lambda x: tokenizer2(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS2, lemmatize=True))
    return df

# Applying all cleaning data text process to our dataset 
df = clean_sentences(df)

# Creating tokes from our cleaned dataset
tokens = df['sentence'].str.split()
tokens = [word for sublist in tokens for word in sublist]

Cleaning sentences...


# Tokenizing and Padding

In [19]:
# Tokenizing our text data
tokenizer = Tokenizer(oov_token='<oov>') 
tokenizer.fit_on_texts(tokens)
total_words = len(tokenizer.word_index) + 1

In [20]:
# Creating input sequences
input_sequences = []
for line in tokens:
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print("Total input sequences: ", len(input_sequences))

Total input sequences:  5277


In [22]:
# Setting max sequence len for each input sequences
max_sequence_len = max([len(x) for x in input_sequences])

In [23]:
# Padding input sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Creating Feature and Label

In [24]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Creating Model

In [35]:
model = Sequential()
model.add(Embedding(total_words, 32, input_length=max_sequence_len-1))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128,kernel_regularizer=tf.keras.regularizers.l2(0.001),recurrent_regularizer=tf.keras.regularizers.l2(0.001))))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.001)))

# Setting Up Costumized Adam Optimizers

In [36]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.95)
optimizers = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compiling Model

In [37]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers, metrics=['accuracy'])

# Training Model

In [42]:
history = model.fit(xs, ys, epochs=1000, validation_split=0.1, verbose=1)
print(model)

Epoch 1/1000
149/149 [==============================] - 4s 30ms/step - loss: 2.3243 - accuracy: 0.6555 - val_loss: 2.4780 - val_accuracy: 0.7670
Epoch 2/1000
149/149 [==============================] - 5s 32ms/step - loss: 2.2716 - accuracy: 0.6679 - val_loss: 2.3639 - val_accuracy: 0.7569
Epoch 3/1000
149/149 [==============================] - 4s 29ms/step - loss: 2.2624 - accuracy: 0.6572 - val_loss: 2.3975 - val_accuracy: 0.7557
Epoch 4/1000
149/149 [==============================] - 4s 30ms/step - loss: 2.2758 - accuracy: 0.6677 - val_loss: 2.3666 - val_accuracy: 0.7595
Epoch 5/1000
149/149 [==============================] - 4s 29ms/step - loss: 2.2347 - accuracy: 0.6702 - val_loss: 2.3790 - val_accuracy: 0.7342
Epoch 6/1000
149/149 [==============================] - 5s 32ms/step - loss: 2.2513 - accuracy: 0.6633 - val_loss: 2.2813 - val_accuracy: 0.7872
Epoch 7/1000
149/149 [==============================] - 5s 33ms/step - loss: 2.2259 - accuracy: 0.6679 - val_loss: 2.2626 - val_ac

# Simulation

In [43]:
seed_text = "java"
seed_text_sentence=seed_text
next_words = 5
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text_sentence += " " + output_word
    seed_text = output_word
print(seed_text_sentence)

1/1 [==============================] - 0s 22ms/step
java dicoding 18 jan modules dicoding


In [46]:
seed_text = "dropdown error"
next_words = 1
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 19ms/step
dropdown error dicoding


In [47]:
seed_text = "kuis bermasalah"
next_words = 5
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 19ms/step
kuis bermasalah dicoding 18 jan 2021 24


**This next word that's been predicted looked "not fine", here's the reasons:**



1.   Our model's accuracy is not reach at least 80%.
2.   Limited good dataset that used to trained.
3.   This nlp model tricky enough, adding new data not always make model better, because that data may be not necessarily good enough.



# Savel Model in h5 Format and Downloading json File

In [44]:
import json
model.save('model.h5')
with open('tokenizer.json', 'w') as f:
  f.write(json.dumps(tokenizer.to_json()))